# Convergence and Limit Theorems

Author & Instructor: Diana NURBAKOVA, PhD.

In [ ]:
%%html
<link rel="stylesheet" type="text/css" href="../styles/styles.css">

## Learning Objectives

By the end of this lesson, you will be able to:
- Distinguish between different types of convergence (in distribution, in probability, almost surely) and explain when each type matters in practice
- Explain intuitively why the Law of Large Numbers guarantees that sample means converge to population means, and identify when this guarantee 
- Understand why the Central Limit Theorem is remarkable (universality of normality) and explain its fundamental role in statistical inference
- Calculate required sample sizes for desired precision using CLT-based formulas: $n = (z_{\alpha/2}\cdot\sigma/\varepsilon)^2$

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings('ignore')

# Set style for better-looking plots
plt.style.use('seaborn-v0_8-darkgrid')
#sns.set_palette("husl")

# Set random seed for reproducibility
np.random.seed(42)

In [ ]:
import sys
from pathlib import Path

# Add the "resources" directory to the path
project_root = Path().resolve().parent
resources_path = project_root / 'resources'
sys.path.insert(0, str(resources_path))

In [ ]:
from limit_theorems import (show_transformation, demo_clt_cauchy, example_no_convergence, example_convergence_in_distribution_only, 
                            example_almost_sure_convergence, example_convergence_in_probability,
                            casino_simulation_intro, bridge_to_formal_definition, polling_simulation)

<div class="alert alert-info">
<h4>🎯 Today's Challenge: The Monte Carlo Reliability Crisis</h4>
<p><strong>Scenario:</strong> You're building a fraud detection AI system. Your model outputs a probability score, but to get the final prediction, you need to run Monte Carlo sampling to account for model uncertainty.</p>

<p><strong>Your manager asks:</strong> "How many Monte Carlo samples do we need? Each sample costs us 10ms of compute time."</p>

<p><strong>You try different sample sizes:</strong></p>
<ul>
<li><strong>10 samples (100ms):</strong> Estimate = 0.73</li>
<li>Run again: 0.61</li>
<li>Run again: 0.84</li>
<li>Run again: 0.55</li>
</ul>

<p><strong>1,000 samples (10 seconds):</strong> Estimate = 0.698, then 0.702, then 0.695</p>

<p><strong>100,000 samples (16 minutes):</strong> Estimate = 0.7001, then 0.7003, then 0.6999</p>

<p><strong>The Million Dollar Questions:</strong></p>
<ol>
<li>Will we EVER get <em>exactly</em> 0.7000? Or are we chasing an impossible dream?</li>
<li>How fast does the "jumpiness" decrease? Is there a mathematical pattern?</li>
<li>Can we <em>guarantee</em> we're within 0.01 of the true value? With how many samples?</li>
<li>Your manager needs an answer in under 1 second (max 100 samples). What do you tell them?</li>
</ol>

<p><strong>Make your predictions now.</strong> By the end of today, you'll be able to answer all of these with mathematical precision.</p>
</div>

In [ ]:
def demo_estimating_prob_hook(true_prob=0.7):
    np.random.seed(42)

    print("Live Simulation: Estimating probability = 0.7\n")

    for n_samples in [10, 50, 100, 500, 1000, 5000]:
        estimates = []
        # run 5 trials
        for trial in range(5):
            estimate = np.random.binomial(n_samples, true_prob) / n_samples
            estimates.append(estimate)
        
        print(f"\nWith {n_samples:5d} samples:")
        print(f"  Estimates: {[f'{e:.3f}' for e in estimates]}")
        print(f"  Range: {max(estimates) - min(estimates):.3f}")
        print(f"  Std Dev: {np.std(estimates):.4f}")

demo_estimating_prob_hook()

Note that if we consider the range (i.e. the difference between the max and the min elements), we can notice that the "*jumpiness*" is decreasing with the increase of the number of samples. The question that remains is *how fast*?

## Types of Convergence

In daily life, we say things "*converge*" loosely. In probability theory, we need precision.

Look at these three sequences approaching 0:

- Sequence A: 1, 0.5, 0.25, 0.125, ... (definitely reaching 0)
- Sequence B: 1, 0, 1, 0, 1, 0, ... (oscillating, never settles)
- Sequence C: 1, 0.5, 0.25, 100, 0.001, 0.0005, ... (mostly approaching, rare spikes)

> Which ones "converge to 0"? It depends on what kind of convergence we mean.


In [ ]:
# Define the three sequences
n_terms = 50

# Sequence A: 1, 0.5, 0.25, 0.125, ... (geometric: 1/2^n)
# Definitely converges to 0
sequence_A = np.array([1 / (2**n) for n in range(n_terms)])

# Sequence B: 1, 0, 1, 0, 1, 0, ... (oscillating)
# Never settles
sequence_B = np.array([1 if n % 2 == 0 else 0 for n in range(n_terms)])

# Sequence C: Mostly approaching 0, but with rare spikes
# Create base sequence that decreases
sequence_C = np.array([1 / (2**n) for n in range(n_terms)])
# Add rare spikes at specific positions
spike_positions = [15, 32, 47]  # Positions where spikes occur
for pos in spike_positions:
    if pos < n_terms:
        sequence_C[pos] = 100 / (pos + 1)  # Spike that decreases with position


In [ ]:
# Create visualization
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Plot Sequence A
ax1 = axes[0]
ax1.plot(range(n_terms), sequence_A, 'bo-', linewidth=2, markersize=6, alpha=0.7)
ax1.axhline(0, color='red', linestyle='--', linewidth=2, label='Limit = 0')
ax1.fill_between(range(n_terms), 0, sequence_A, alpha=0.3, color='blue')
ax1.set_xlabel('n', fontsize=12, fontweight='bold')
ax1.set_ylabel(r'$a_n$', fontsize=12, fontweight='bold')
ax1.set_title('Sequence A: 1, 1/2, 1/4, 1/8, ...\nDefinitely Converges to 0', 
              fontsize=13, fontweight='bold', color='green')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.set_ylim([-0.1, 1.2])

# Add annotation
ax1.annotate('Smooth decrease\nto zero',
            xy=(25, sequence_A[25]), xytext=(35, 0.5),
            arrowprops=dict(arrowstyle='->', color='green', lw=2),
            fontsize=10, color='green', fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))

# Plot Sequence B
ax2 = axes[1]
ax2.plot(range(n_terms), sequence_B, 'ro-', linewidth=2, markersize=6, alpha=0.7)
ax2.axhline(0.5, color='orange', linestyle='--', linewidth=2, 
            label='Average = 0.5', alpha=0.7)
ax2.set_xlabel('n', fontsize=12, fontweight='bold')
ax2.set_ylabel(r'$b_n$', fontsize=12, fontweight='bold')
ax2.set_title('Sequence B: 1, 0, 1, 0, 1, 0, ...\n Oscillates Forever (No Limit)', 
              fontsize=13, fontweight='bold', color='red')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)
ax2.set_ylim([-0.2, 1.3])

# Add annotation
ax2.annotate('Keeps jumping\nbetween 0 and 1',
            xy=(30, 1), xytext=(35, 1.15),
            arrowprops=dict(arrowstyle='->', color='red', lw=2),
            fontsize=10, color='red', fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='#ffebee', alpha=0.8))

# Plot Sequence C
ax3 = axes[2]
ax3.plot(range(n_terms), sequence_C, 'go-', linewidth=2, markersize=6, alpha=0.7)
ax3.axhline(0, color='red', linestyle='--', linewidth=2, label='Limit = 0?')

# Highlight the spikes
for pos in spike_positions:
    if pos < n_terms:
        ax3.plot(pos, sequence_C[pos], 'r*', markersize=20, label='Spike!' if pos == spike_positions[0] else '')

ax3.set_xlabel('n', fontsize=12, fontweight='bold')
ax3.set_ylabel(r'$c_n$', fontsize=12, fontweight='bold')
ax3.set_title('Sequence C: Mostly Decreasing, Rare Spikes\nMostly Approaching 0, But...', 
              fontsize=13, fontweight='bold', color='orange')
ax3.legend(fontsize=10)
ax3.grid(True, alpha=0.3)
ax3.set_ylim([-0.5, 8])

# Add annotations for spikes
ax3.annotate('Unexpected\nspike!',
            xy=(spike_positions[0], sequence_C[spike_positions[0]]), 
            xytext=(spike_positions[0] - 8, 5),
            arrowprops=dict(arrowstyle='->', color='red', lw=2),
            fontsize=10, color='red', fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='#fff8e1', alpha=0.8))

plt.suptitle('Three Types of Sequence Behavior: Which Ones "Converge"?', 
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()


<div class="alert alert-success">
<h4>Definition: Convergence in Distribution (or in Law)</h4>

A sequence of random variables $X_1, X_2, ...$ with cumulative distribution functions $F_1, F_2, ...$ **converges in distribution** (or **in law**) to a random variable $X$ with cumulative distribution function $F$ if:

$$\lim_{n \to \infty} F_n(x) = F(x)$$

for all $x$ where $F$ is continuous.

We denote this as:
$$X_n \xrightarrow[n\to \infty]{\mathcal{d}} X$$

*Intuition*: The cumulative distribution functions of $X_n$ converge point-wise to the cumulative distribution function of $X$ at all continuity points of $F$. 
"$X_n$ **behaves like** $X$ for large $n$"

</div>

<div class="alert alert-success">
<h4>Definition: Convergence in Probability</h4>

A sequence of random variables $X_1, X_2, ...$ **converges in probability** to a random variable $X$ if for any $\epsilon > 0$:

$$\lim_{n \to \infty} \mathbb{P}(|X_n - X| > \epsilon) = 0$$

We denote this as:
$$X_n \xrightarrow[n\to \infty]{\mathbb{P}} X$$

*Intuition:* As $n$ increases, the probability that $X_n$ is far from $X$ (by more than $\epsilon$) tends to zero. In other words, $X_n$ gets arbitrarily close to $X$ with increasingly high probability. "$X_n$ is **probably** close to $X$ for large $n$"

*Remark*: Convergence in distribution is weaker than convergence in probability. If $X_n \xrightarrow[n\to \infty]{\mathbb{P}} X$, then $X_n \xrightarrow[n\to \infty]{\mathcal{d}} X$.

</div>

<div class="alert alert-success">
<h4>Definition: Almost Sure Convergence </h4>

Let $X_1, X_2, ...$ be a sequence of i.i.d. random variables with finite expectation $\mathbb{E}(X_i) = \mu$.

Let $\overline{X}_n = \frac{1}{n}\sum_{i=1}^n X_i$ be the sample mean. 

The sequence converges **almost surely** to $X$ if:

$$\mathbb{P}\left(\lim_{n \to \infty} \overline{X}_n = \mu\right) = 1$$

This is also written as:
$$\overline{X}_n \xrightarrow[n\to \infty]{a.s.} \mu$$

where "a.s." stands for "almost surely" (convergence with probability 1).

*Intuition:* The actual sequence converges (except on a set of probability 0). "If you run the experiment, $X_n$ **will** converge to $X$ (almost certainly)"

</div>

When we say "*all paths converge*", we're talking about individual realizations of the random sequence. A "*path*" is one complete run of our random experiment from start to finish. 

Suppose you're estimating the probability of heads by flipping a coin repeatedly:

1. Path 1: You flip and get: H, T, H, H, T, T, H, H, H, T, ...
- Running averages: 1.00, 0.50, 0.67, 0.75, 0.60, 0.50, 0.57, 0.625, 0.67, 0.60, ...

2. Path 2: Your friend flips (different random outcomes): T, T, H, T, H, H, T, H, T, H, ...
- Running averages: 0.00, 0.00, 0.33, 0.25, 0.40, 0.50, 0.43, 0.50, 0.44, 0.50, ...

3. Path 3: Another person flips: H, H, H, T, T, H, T, H, H, H, ...
- Running averages: 1.00, 1.00, 1.00, 0.75, 0.60, 0.67, 0.57, 0.625, 0.67, 0.70, ...

Each sequence of flips is a *path* or *sample path* or *realization* or *trajectory*.

If we compare convergence in probability and almost sure convergence:

1. Convergence in probability says: "For large $n$, MOST paths will be close to 0.5"
- Some paths might wander away temporarily
- We only care that the probability of being far is small

2. Almost sure convergence says: "If you actually run the experiment, YOUR specific path WILL converge to 0.5"
- Not just "probably" - it actually happens
- Only fails on a set of measure zero (impossible events)

> If bad events keep happening, but become rarer and rarer, will they eventually stop happening?
</br>

Consider an analogy of the casino. We are tracking "bad days":
- Day 1: Probability of losing = 0.5
- Day 10: Probability of losing = 0.1
- Day 100: Probability of losing = 0.01
- Day 1000: Probability of losing = 0.001

*Question*: Will you eventually stop having losing days?

Borel-Cantelli says: It depends on whether the sum of probabilities converges:

- If $0.5 + 0.1 + 0.01 + 0.001 + ...$ converges (finite sum) → You'll only have finitely many losing days
- If the sum diverges (infinite) → You'll have infinitely many losing days

<div class="alert alert-success">
<h4>Definition: Borel-Cantelli Lemma</h4>

Let $A_1, A_2, A_3, ...$ be a sequence of events.

If $\sum P(A_n) < \infty$ (the sum of probabilities converges), then $P(A_n\text{ occurs infinitely often}) = 0$

In other words: With probability 1, only finitely many of the events $A_n$ will occur.

Notation: $P(A_n \text{ i.o.}) = 0$ where "*i.o.*" means "*infinitely often*"

Formally: $A_n \text{ i.o.} = \cap_{m=1}^{\infty} \cup_{n=m}^{\infty}A_n$ (Events that occur for infinitely many $n$)
</div>

**Connection to Almost Sure Convergence**

How We Use It:

To prove $X_n \rightarrow X$ almost surely, we define "bad events":
$A_n = {|X_n - X| > \varepsilon}$  (the event that we're more than ε away from the limit)

Goal: Show that $P(A_n \text{ i.o.}) = 0$

That is, we want to show: "With probability 1, we're only far from $X$ finitely many times, and then we stay close forever."

**Borel-Cantelli Strategy:**

1. Show that $Σ P(|X_n - X| > \varepsilon) < \infty$
2. Conclude that $P(|X_n - X| > \varepsilon \text{ infinitely often}) = 0$
3. Therefore, eventually $|X_n - X| \leq \varepsilon$ for all sufficiently large $n$ (with probability 1)
4. This holds for every $\varepsilon > 0$, so $X_n \rightarrow X$ almost surely

<a id="borel-cantelli-ex"></a>
<div class="alert alert-example">
<h4>Calculated Example: Almost Sure Convergence and Borel-Cantelli Lemma</h4>

Let $Z_1, Z_2, Z_3, ...$ be i.i.d. with $E[Z_i] = 0$ and $Var(Z_i) = \sigma^2 = 1 < \infty$.

Let $X_n = \sum_{i=1}^n Z_i/n^2$.

1. Step 1: Define bad event

$$A_n = {|X_n| > \varepsilon}$$

2. Step 2: Bound probability using Chebyshev's inequality

$$P(|X_n| > \varepsilon) \leq \frac{Var(X_n)}{\varepsilon^2}$$

Let's find the variance of $X_n$:
$$Var(X_n) = Var\bigg(\sum_{i=1}^n Z_i/n^2\bigg) = 1/n^4 Var\bigg(\sum_{i=1}^n Z_i\bigg) = [\text{by indep.}] = n/n^4\times Var(Z_1) = 1/n^3$$
So $P(|X_n| > \varepsilon) \leq \frac{1}{n^3\varepsilon^2}$

3. Step 3: Check if sum converges

$$\sum_{n=1}^\infty P(|X_n| > \varepsilon) \leq \sum_{n=1}^\infty \frac{1}{n^3\varepsilon^2} = (1/\varepsilon^2)\times \sum_{n=1}^\infty \frac{1}{n^3}$$

The series $\sum_{n=1}^\infty \frac{1}{n^3}$ [converges to a finite value](https://en.wikipedia.org/wiki/Convergence_tests#Examples) ($p$-series with $p=3 > 1$; intuitive sense: when $p$ is large, $1/n^p$ becomes tiny very quickly, so we're essentially just adding $1 + \text{(tiny terms)}$), so the sum is finite.

4. Step 4: Apply Borel-Cantelli

Since $\sum_{n=1}^\infty P(A_n) < \infty$, we have $P(A_n \text{ i.o.}) = 0$

Therefore $P(X_n \rightarrow 0) = 1$, i.e. $X_n$ converges almost surely to 0.


</div>

### Example Scenarios

We'll explore four scenarios:

1. No Convergence: Sequence stays random forever
2. Convergence in Distribution Only: CDFs converge, but values don't
3. Convergence in Probability + Distribution: Values probably get close
4. Almost Sure + Probability + Distribution: Values actually converge


<div class="alert alert-example">
<h4>Scenario 1: No Convergence - The Oscillating Sequence</h4>

Consider  
$$X_n = (-1)^n \cdot n$$


*Behavior*: $X_1 = -1, X_2 = 2, X_3 = -3, X_4 = 4, X_5 = -5, ...$

*Mathematical Analysis:*

- Does NOT converge to any value: oscillates and grows
- Not convergent in distribution:> CDFs don't stabilize
- Not convergent in probability: $P(|X_n - X| < \varepsilon)$ doesn't go to 1 for any $X$
- Not almost surely convergent: the sequence diverges


*Why it fails:* No matter what value you pick as a "limit," the sequence keeps moving away from it.
</div>


<details>
<summary>Reveal mathematical proof</summary>

Our claim: $X_n$ does not converge to any $X$

Let's apply the strategy of proof by contradiction.

Suppose $X_n \rightarrow X$ for some $X \in \mathbb{R}$. 

Then for $\varepsilon = 1$:
    $P(|X_n - X| < 1)\text{ should} \rightarrow 1$
    
But:
- For even $n$: $X_n = n \rightarrow +\infty$
- For odd $n$:  $X_n = -n \rightarrow -\infty$
    
So $|X_n - X| \rightarrow \infty$ for ANY $X$.
    
Therefore: NO convergence (not in distribution, not in probability, not almost surely)

**Numerical evidence:**

|$X_i$|Distance from 0|
|:--:|:--:|
|$X_5 = -5$ |  5  |
|$X_{10} = 10$ |  10 |
|$X_{20} = 20$ |  20 |
|$X_{50} = 50$ |  50 |

*Key Observations*
1. Sign alternates: Values flip between positive and negative
2. Magnitude grows: The absolute values keep increasing
3. No stabilization: Distance from any proposed limit increases without bound
4. Pattern persists: No matter how large $n$ gets, the oscillation and growth continue

*Interpretation*:

This sequence does NOT converge. It violates the most basic requirement: the values don't approach any fixed number. This is a sign of:
- Unstable process
- In ML context: divergent training, exploding gradients
- Action needed: fundamental fix required (reduce learning rate, add clipping, check for bugs)

</details>

In [ ]:
# numeric verification
for n in [5, 10, 20, 50]:
    val = ((-1) ** n) * n
    print(f"X_{n:2d} = {val:6.0f}   |  Distance from 0: {abs(val):6.0f}")


*Practical interpretation:*
- This is like a training process that's unstable
- If $X_n$ is a metric, it oscillates wildly and never settles
- In ML: divergent training, need to fix hyperparameters (or check for bugs)
- In statistics: an estimator that doesn't converge is useless

In [ ]:
# visualisation
example_no_convergence()

<div class="alert alert-example">
<h4>Scenario 2: Convergence in Distribution ONLY</h4>

Consider:
$$X_n = Z \cdot (-1)^n\text{, where } Z \sim N(0,1)$$

*Behavior*: ${X_n}$ flips sign each time, but maintains same distribution

*Mathematical Analysis:*
- ✓ Converges in distribution: $X_n \sim N(0,1)$ for all $n$ (same CDF!)
- ✗ Does NOT converge in probability: $P(|X_n - X_{n+1}| < \varepsilon) = P(|2Z| < \varepsilon) \neq 1$
- ✗ Does NOT converge almost surely: sequence oscillates forever
</ul>

*Key Insight*: The *distribution* stays the same, but individual *values* keep jumping around.

</div>

<details>
<summary>Reveal mathematical proof</summary>

1. First, let's check convergence in distribution
    
For all $n$: $X_n = Z \cdot (-1)^n$ where $Z \sim N(0,1)$
    
Since $(-1)^n \in {-1, 1}$ and $Z \sim N(0,1)$:
- Both $Z$ and $-Z$ have same distribution
- Therefore $X_n \sim N(0,1)$ for all $n$
    
In this case, $X_n$ has CDF of standard normal distribution: 

$F_n(x) = P(X_n \leq x) = \Phi(x)$
    
    
Hence: $\lim\limits_{n\to +\infty}F_n(x) = \Phi(x)$ for all $n$

So: $X_n  \xrightarrow[n\to+\infty]{d} X \sim N(0,1)$

$X_n$ converges in distribution to $X\sim N(0,1)$
    
2. Second, let's check convergence in probability
    
$$X_{n+1} - X_n = Z·(-1)^{n+1} - Z·(-1)^n = -2Z·(-1)^n$$
    
$$|X_{n+1} - X_n| = 2|Z|$$
    
$$P(|X_{n+1} - X_n| < \varepsilon) = P(|Z| < \varepsilon/2) \neq 1\text{ as } n \rightarrow \infty$$
    
So, we note that the values keep jumping.

Therefore, there is no convergence in probability.

**Numerical Verification:**

Let $Z = 0.496714$

|$X_i$ |oscillates between ±0.496714|
|:--:|:--:|
|$X_1 = -0.496714$   | YES | 
|$X_2 = 0.496714$   | YES |
|$X_3 = -0.496714$   | YES |
|$X_4 = 0.496714$   | YES |
|$X_5 = -0.496714$   | YES |
|$X_{10} = 0.496714$   | YES |
|$X_{50} = 0.496714$   | YES |
|$X_{100} = 0.496714$   | YES |

Key Observations:
1. Perfect oscillation: Values jump between exactly +Z and -Z
2. No trend toward zero: Distance remains constant forever
3. Distribution is stable: If you collect many samples at any n, you get N(0,1)
4. Individual values don't converge: Each path keeps oscillating indefinitely

*Interpretation*:

Distribution converges, but values don't. This tells us:
- The "average behavior" is predictable (distribution is $N(0,1)$)
- But individual realizations are unreliable
- In ML context: Your model's uncertainty distribution is well-calibrated, but individual predictions are noisy
- Solution: Average multiple predictions, or decrease learning rate to get value convergence

</details>
    

In [ ]:
# numerical verification
np.random.seed(42)
# Generate sequence - ONE realization
Z = np.random.normal(0, 1)  # Draw once
print(f"Z = {Z:.6f}")
for n in [1, 2, 3, 4, 5, 10, 50, 100]:
    val = Z * ((-1) ** n)
    print(f"X_{n:3d} = {val:8.6f}   (oscillates between ±{abs(Z):.6f})")

Practical interpretation:
- ✓ Distribution convergence: 'On average, behavior is stable'
    * Example: Your model's prediction distribution stays constant
    * Example: Error bars remain the same size
- ✗ No value convergence: 'Individual predictions still jump around'
    * Example: Predictions oscillate even though distribution is stable
    * Example: SGD with constant high learning rate - loss distribution stays the same but actual loss values keep jumping
- When you see this in ML:
    * Model uncertainty quantification: distribution is calibrated 
    * But individual predictions are unreliable (high variance)
    * Need to average multiple predictions for stability

In [ ]:
# visualisation
example_convergence_in_distribution_only()

<div class="alert alert-example">
<h4>Scenario 3: Convergence in Probability</h4>

Consider:
$X_n = Z / n \text{, where } Z \sim N(0,1)$

*Behavior*: ${X_n} \rightarrow 0$ as $n \rightarrow \infty$

*Mathematical Analysis:*
- ✓ Converges in probability: $P(|X_n - 0| > \varepsilon) = P(|Z| > n\varepsilon) \rightarrow 0$
- ✓ Converges in distribution: $X_n \xrightarrow[n\to+\infty]{d} \delta_0$ (point mass at 0)
- ✗ Does NOT converge almost surely: Individual paths may oscillate

</ul>

*Key Insight*: For large $n$, $X_n$ is *probably* close to 0, but there's always a small chance it's not (because $Z$ could be large).

</div>

<details>
<summary>Reveal mathematical proof</summary>

1. First, let's focus on the convergence in probability
    
  What we have: $X_n = Z/n$ where $Z \sim N(0,1)$
    
  For any $\varepsilon > 0$:
  $$P(|X_n - 0| > \varepsilon) = P(|Z/n| > \varepsilon) = P(|Z| > n\varepsilon) = 2·P(Z > n\varepsilon) = 2·[1 - \Phi(n\varepsilon)] \rightarrow 0 \text{ as } n \rightarrow \infty$$
    
  Since $\Phi(n\varepsilon) \rightarrow 1$ as $n \rightarrow \infty$, therefore: $X_n \xrightarrow[n \rightarrow \infty]{p} 0$ 
   
2. Second, let's check convergence in distribution
    
  Proof:
  Xₙ ~ N(0, 1/n²)
    
  CDF: Fₙ(x) = Φ(x·n)
  - For x < 0: Fₙ(x) = Φ(x·n) → 0
  - For x > 0: Fₙ(x) = Φ(x·n) → 1
  - For x = 0: Fₙ(0) = 0.5 for all n
    
  Limit is point mass at 0: δ₀
  Therefore: Xₙ →ᵈ δ₀ ✓
    
  Note: Convergence in probability ⟹ Convergence in distribution

*Numerical Verification*

Let $Z = 2.5$ (a particular realization)

| $X_i$ | $P(|X_n| > 0.1)$|
|:--:|:--:|
| $X_1 =  2.50000$   |    0.920344 |
|$X_2 =  1.25000$   |   0.841481|
|$X_5 =  0.50000$   |   0.617075|
|$X_{10} =  0.25000$   |   0.317311|
|$X_{20} =  0.12500$   |   0.045500|
|$X_{50} =  0.05000$   |   0.000001|
|$X_{100} =  0.02500$   |   0.000000|
| $X_{200} =  0.01250$   |   0.000000|

*Key Observations*
1. Monotonic decrease: Values consistently get smaller
2. Approaching zero: Clear trend toward 0
3. Predictable rate: Each doubling of $n$ halves the value ($1/n$ pattern)
4. Not quite zero: Never exactly reaches 0, but gets arbitrarily close
5. Depends on $Z$: If $Z$ is large, takes longer to get close to 0

*Interpretation*

Values probably get close to 0. This means:
- For large $n$, $X_n$ is likely very close to 0
- But there's always a small chance it's not (if $Z$ happens to be large)
- Different random draws of $Z$ give different paths, but all trend toward 0
- In ML context: Your algorithm usually converges, good enough for practice
- The specific path depends on random initialization ($Z$), but outcome is predictable

</details>

In [ ]:
# Numerical Verification
print("Drawing Z = 2.5 (a particular realization):")
Z = 2.5
for n in [1, 2, 5, 10, 20, 50, 100, 200]:
    val = Z / n
    prob_outside = 2 * (1 - stats.norm.cdf(n * 0.1))  # P(|Xₙ| > 0.1)
    print(f"X_{n:3d} = {val:8.5f}   |  P(|Xₙ| > 0.1) = {prob_outside:.6f}")

In [ ]:
print("Drawing Z = 500 (a particular realization):")
Z = 500
for n in [1, 2, 5, 10, 20, 50, 100, 200]:
    val = Z / n
    prob_outside = 2 * (1 - stats.norm.cdf(n * 0.1))  # P(|Xₙ| > 0.1)
    print(f"X_{n:3d} = {val:8.5f}   |  P(|Xₙ| > 0.1) = {prob_outside:.6f}")

*Real-world examples:*
1. Sample mean with fixed data:
$\bar{X_n} = (X_1 + ... + X_n)/n \rightarrow \mu$ (Law of Large Numbers)
    
2. Regularization in ML:
- $Loss_n = MSE + \lambda/n·||w||^2 \rightarrow MSE$ as $n\rightarrow \infty$ (regularization term $\lambda/n·||w||^2$ vanishes)
    
3. Learning rate decay:
- $\theta_{n+1} = \theta_n - (1/n)\cdot\nabla L(\theta_n)$
- Step sizes → 0, likely to converge
    
4. Monte Carlo estimates:vill
- Estimate with $n$ samples → true value
    
*What it does NOT guarantee:*
- Individual sequences may still have occasional jumps
- Not every realization converges (just 'most' do)
- Requires infinite samples for perfect convergence

*Practical Interpretation:*
    
Convergence in probability means:
- For large $n$, $X_n$ is PROBABLY very close to the limit
- The probability of being far away goes to zero
- But there's always a tiny chance of deviation (if $Z$ is large)

In [ ]:
# visualisation
example_convergence_in_probability()

<div class="alert alert-example">
<h4>Scenario 4: Almost Sure Convergence (Strongest Type)</h4>

Consider:
$X_n = \sum_{i=1}^\infty Z_i / n^2 \text{, where } Z_i \sim N(0,1) \text{ i.i.d.}$

*Behavior*: ${X_n} \rightarrow 0$ almost surely (with probability 1)

*Mathematical Analysis:*
- ✓ Converges almost surely: $P(X_n \rightarrow 0) = 1$
- ✓ Converges in probability: implied by a.s. convergence
- ✓ Converges in distribution: implied by convergence in probability
</ul>

*Key Insight*: The actual sequence values converge to 0 (not just probabilistically, but the paths themselves).

</div>

<details>
<summary>Reveal mathematical proof</summary>

We'd like to prove almost sure convergence.

Proof outline (detailed proof has been discussed in the section [Borel-Cantelli Lemma](#borel-cantelli-ex)):
    
Let $X_n = (Z_1 + Z_2 + ... + Z_n)/n^2$ where $Z_i \sim N(0,1)$.
    
The expected value $E[X_n] = E[(Z_1 + Z_2 + ... + Z_n)/n^2] = n/2^2E[Z_i] =  0$

Variance:     
$$Var(X_n) = Var\bigg(\sum_{i=1}^n Z_i/n^2\bigg) = 1/n^4 Var\bigg(\sum_{i=1}^n Z_i\bigg) = [\text{by indep.}] = n/n^4\times Var(Z_1) = 1/n^3$$

We can bound the probability of $P(|X_n| > \varepsilon)$ using Chebyshev's inequality $\varepsilon > 0$:
    $$P(|X_n| > \varepsilon) ≤ Var(X_n)/\varepsilon^2 = 1/(n^3\varepsilon^2)$$
    
Key point: $$\sum_{n=1}^\infty P(|X_n| > \varepsilon) \leq \sum_{n=1}^\infty 1/(n^3\varepsilon^2) = (1/\varepsilon^2)\cdot \sum_{n=1}^\infty 1/n^3 < \infty \text{ (p-series, p=3 > 1)}$$
    
By Borel-Cantelli Lemma:
    $P(|X_n| > \varepsilon \text{ infinitely often}) = 0$
    
Therefore: $P(X_n \rightarrow 0) = 1$
    
This implies:
- Convergence in probability
- Convergence in distribution
- Convergence almost surely

**Numerical verification:**

| $X_i$ | Realization 1| Realization 2 | Realization 3 | Realization 4 | Realization 5 |
|:---:|:---:|:---:|:---:|:---:|:---:|
| $X_{10}$   | 0.0448061   | -0.0137659 | 0.0487307 | 0.0199982  | -0.0281947 |
| $X_{50}$   |  -0.0045095 | -0.0051722 | 0.0065431 | 0.0002167  | -0.0029837 |
|  $X_{100}$ | -0.0010385  | -0.0011531 | 0.0016017 | -0.0009915 | -0.0011160 |
|  $X_{200}$ | -0.0002039  | -0.0002280 | 0.0006682 | 0.0001646  | -0.0003293 |
|  $X_{500}$ |  0.0000137  | 0.0000637  | 0.0002170 | 0.0000664  | -0.0000230 |

*Key Observations*: 

1. All paths converge: Every realization trends toward 0
2. Fast convergence: Values become tiny very quickly
3. Different paths, same destination: Each realization takes a different route but all reach 0
4. Accelerating convergence: The rate of decrease speeds up (due to n² in denominator)

*Interpretation*

Actually converges to 0. This is the strongest guarantee:

- Not just "probably" - the paths themselves converge
- Every run will reach the limit (with probability 1)
- Convergence is fast and reliable
- In ML context: Training will definitely converge, can trust stopping criteria
- Different random seeds all lead to the same outcome (convergence to optimum)

</details>



In [ ]:
np.random.seed(42)
    
# Numerical verification
n_max = 500
# Generating 5 independent realizations:
for trial in range(5):
    Z = np.random.normal(0, 1, n_max)
    cumsum_Z = np.cumsum(Z)
        
    print(f"\nRealization {trial + 1}:")
    for n in [10, 50, 100, 200, 500]:
        if n <= n_max:
            val = cumsum_Z[n-1] / (n ** 2)
            print(f"  X_{n:3d} = {val:10.7f}")

*Practical Interpretation:*
- Almost sure convergence is the STRONGEST type:
- The actual sequence WILL converge (with probability 1)
- Not just 'probably' close, but paths actually reach the limit
- Only excludes a set of probability 0 (pathological cases)

*Real-world examples:*
1. Stochastic Gradient Descent with decreasing learning rate:
    
If $\eta_n = 1/n^2$ and $\sum\eta_n^2 < \infty$, then $\theta_n \xrightarrow[n\to+\infty]{a.s.} \theta^*$ ([Robbins-Monro conditions](https://en.wikipedia.org/wiki/Stochastic_approximation))

2. Sample mean (Strong Law of Large Numbers):
    
$\bar{X_n} = (X_1 + ... + X_n)/n \xrightarrow[n\to+\infty]{a.s.}\mu$, i.e. your estimate WILL converge to true mean
    
3. Monte Carlo integration with proper variance control:
    
$\int f(x)dx$ estimated by $(1/n)\sum f(X_i) \xrightarrow[n\to+\infty]{a.s.}\text{ true integral}$

4. Online learning with vanishing step sizes:
    
Parameter updates converge to optimal value
    

*Why it matters:*
- Strongest theoretical guarantee
- Individual runs will converge (not just on average)
- Required for many theoretical proofs in ML
- Justifies stopping criteria in optimization

In [ ]:
# visualisation
example_almost_sure_convergence(py=False)

**Hierarchy of convergence:**
  
$$\text{Almost Sure} \Rightarrow \text{In Probability} \Rightarrow \text{In Distribution}$$

(Each arrow is strict: $\Leftarrow$ does NOT hold in general)

<div class="alert alert-info">
<h4>Comparison: Types of Convergence</h4>

<table style="width: 100%; border-collapse: collapse; background: white; margin: 10px 0; font-size: 0.95em;">
<thead>
<tr style="background: #e3f2fd;">
<th style="padding: 12px; border: 1px solid #ccc;">Property</th>
<th style="padding: 12px; border: 1px solid #ccc;">Example 1:<br>No Conv.</th>
<th style="padding: 12px; border: 1px solid #ccc;">Example 2:<br>Dist. Only</th>
<th style="padding: 12px; border: 1px solid #ccc;">Example 3:<br>In Prob.</th>
<th style="padding: 12px; border: 1px solid #ccc;">Example 4:<br>Almost Sure</th>
</tr>
</thead>
<tbody>
<tr>
<td style="padding: 10px; border: 1px solid #ccc;"><strong>Sequence</strong></td>
<td style="padding: 10px; border: 1px solid #ccc;">

$(-1)^n·n$</td>
<td style="padding: 10px; border: 1px solid #ccc;">

$Z·(-1)ⁿ$</td>
<td style="padding: 10px; border: 1px solid #ccc;">

$Z/n$</td>
<td style="padding: 10px; border: 1px solid #ccc;">

$\sum_i Z_i/n^2$</td>
</tr>
<tr style="background: #f5f5f5;">
<td style="padding: 10px; border: 1px solid #ccc;"><strong>Limit (if exists)</strong></td>
<td style="padding: 10px; border: 1px solid #ccc;">None</td>
<td style="padding: 10px; border: 1px solid #ccc;">N(0,1) (dist)</td>
<td style="padding: 10px; border: 1px solid #ccc;">0</td>
<td style="padding: 10px; border: 1px solid #ccc;">0</td>
</tr>
<tr>
<td style="padding: 10px; border: 1px solid #ccc;"><strong>In Distribution?</strong></td>
<td style="padding: 10px; border: 1px solid #ccc; color: #c62828;">❌ No</td>
<td style="padding: 10px; border: 1px solid #ccc; color: #2e7d32;">✓ Yes</td>
<td style="padding: 10px; border: 1px solid #ccc; color: #2e7d32;">✓ Yes</td>
<td style="padding: 10px; border: 1px solid #ccc; color: #2e7d32;">✓ Yes</td>
</tr>
<tr style="background: #f5f5f5;">
<td style="padding: 10px; border: 1px solid #ccc;"><strong>In Probability?</strong></td>
<td style="padding: 10px; border: 1px solid #ccc; color: #c62828;">❌ No</td>
<td style="padding: 10px; border: 1px solid #ccc; color: #c62828;">❌ No</td>
<td style="padding: 10px; border: 1px solid #ccc; color: #2e7d32;">✓ Yes</td>
<td style="padding: 10px; border: 1px solid #ccc; color: #2e7d32;">✓ Yes</td>
</tr>
<tr>
<td style="padding: 10px; border: 1px solid #ccc;"><strong>Almost Surely?</strong></td>
<td style="padding: 10px; border: 1px solid #ccc; color: #c62828;">❌ No</td>
<td style="padding: 10px; border: 1px solid #ccc; color: #c62828;">❌ No</td>
<td style="padding: 10px; border: 1px solid #ccc; color: #c62828;">❌ No</td>
<td style="padding: 10px; border: 1px solid #ccc; color: #2e7d32;">✓ Yes</td>
</tr>
<tr style="background: #f5f5f5;">
<td style="padding: 10px; border: 1px solid #ccc;"><strong>Path Behavior</strong></td>
<td style="padding: 10px; border: 1px solid #ccc;">Diverges</td>
<td style="padding: 10px; border: 1px solid #ccc;">Oscillates forever</td>
<td style="padding: 10px; border: 1px solid #ccc;">Usually converges</td>
<td style="padding: 10px; border: 1px solid #ccc;">Actually converges</td>
</tr>
<tr>
<td style="padding: 10px; border: 1px solid #ccc;"><strong>ML Example</strong></td>
<td style="padding: 10px; border: 1px solid #ccc;">Divergent training</td>
<td style="padding: 10px; border: 1px solid #ccc;">SGD w/ const. high LR</td>
<td style="padding: 10px; border: 1px solid #ccc;">LR decay: 1/n</td>
<td style="padding: 10px; border: 1px solid #ccc;">LR decay: 1/n²</td>
</tr>
<tr style="background: #f5f5f5;">
<td style="padding: 10px; border: 1px solid #ccc;"><strong>Practical Meaning</strong></td>
<td style="padding: 10px; border: 1px solid #ccc;">Unstable, useless</td>
<td style="padding: 10px; border: 1px solid #ccc;">Stable dist., noisy values</td>
<td style="padding: 10px; border: 1px solid #ccc;">Probably close</td>
<td style="padding: 10px; border: 1px solid #ccc;">Actually converges</td>
</tr>
</tbody>
</table>

<p style="background: #fff8e1; padding: 15px; border-radius: 8px; margin: 15px 0;">
<strong>Key Insight:</strong> The hierarchy is strict:<br>
<strong>Almost Sure ⟹ In Probability ⟹ In Distribution</strong><br>
But the reverse implications do NOT hold!
</p>
</div>


### Practical Implications: What Does Each Type Really Mean?

| | No Convergence | Convergence in Distribution | Convergence in Probability | Almost Sure Convergence |
|---|----|---|----|----|
| **What it means** |<ul><li>Your process is unstable or diverging</li><li>No meaningful limit exists</li><li>Cannot make reliable predictions</li></ul> |<ul> <li>The <em>distribution</em> of outcomes is stable</li>    <li>But individual values keep jumping around</li> <li>"On average" behavior is predictable</li> <li>Individual predictions are still noisy </li></ul>| <ul><li>For large n, values are <em>probably</em> close to the limit</li><li>Probability of being far away → 0</li><li>Most runs converge, but occasional deviations possible</li>    <li>Good enough for most practical purposes</li></ul>| <ul><li>The actual sequence <em>will</em> converge (with probability 1)</li><li>Not just "probably close" - paths actually reach the limit</li><li>Strongest possible guarantee</li><li>Only fails on a set of measure zero (impossible events)</li> </ul>|
| **When you see it (ML)** |<ul><li>Training loss oscillating wildly and growing</li><li>Learning rate too high</li><li>Bug in code (gradient explosion)</li><li>Wrong optimization algorithm for the problem</li></ul>| <ul> <li>SGD with constant learning rate: loss distribution stable, but values oscillate</li> <li>Monte Carlo sampling: histogram shape stable, but samples vary</li><li>Ensemble predictions: distribution is calibrated, but individual model outputs vary</li><li>Uncertainty quantification: predictive distribution correct, point estimates noisy</li></ul> |<ul><li>Sample mean converging to population mean (Weak LLN)</li><li>SGD with learning rate decay $\eta_n = 1/n$</li> <li>Monte Carlo estimates with increasing samples</li><li>Stochastic approximation algorithms</li><li>Online learning with diminishing step sizes</li></ul> |<ul><li>Sample mean (Strong LLN): X̄ₙ → μ almost surely</li> <li>SGD with aggressive learning rate decay: $\eta_n = 1/n^2$</li><li>Robbins-Monro stochastic approximation (when $\sum\eta_n^2 < \infty$)</li> <li>Well-designed online learning algorithms</li> <li>Martingale convergence theorems</li> </ul> |
| **What to do**|<ul><li>Stop and debug!</li><li>Reduce learning rate</li><li>Add gradient clipping</li><li>Check for numerical instabilities</li></ul> |<ul><li>For distribution estimation: you're done (if that's your goal)</li><li>For point estimates: average multiple samples</li> <li>Use ensemble methods to reduce variance</li><li>Consider moving to probability or a.s. convergence (e.g., decrease LR)</li></ul> | <ul><li>This is usually sufficient for ML applications</li><li>Can use this to set stopping criteria</li><li>Compute confidence intervals for reliability</li><li>Monitor convergence with validation metrics</li></ul>|<ul><li>Best possible scenario</li><li>Can rely on convergence for theoretical analysis</li><li>Use in proofs of algorithm correctness</li><li>Justifies stopping when change becomes small</li></ul> |

<center>
<img src="img/convergence-decision-tree.svg" alt="Convergence Decision Tree" width="1000px">
</center>

<div class="alert alert-summary">
<h4>🔑 Key Takeaways for ML Practitioners</h4>
    
<ol>
<li><strong>Check which type you have:</strong>
        <ul>
        <li>Plot training curves - do they stabilize?</li>
        <li>Run multiple random seeds - do all converge?</li>
        <li>Monitor variance - does it decrease?</li>
        </ul>
</li>
    
<li><strong>Match convergence type to your needs:</strong>
        <ul>
        <li>Distribution only: OK for uncertainty quantification</li>
        <li>In probability: OK for most ML tasks</li>
        <li>Almost sure: Needed for theoretical guarantees</li>
        </ul>
</li>
    
<li><strong>Tune accordingly:</strong>
        <ul>
        <li>No convergence → reduce LR, add regularization</li>
        <li>Dist. only → decrease LR over time if you need point convergence</li>
        <li>Probability → already good for practice!</li>
        <li>Almost sure → optimal setup for theory</li>
        </ul>
</li>
    
<li><strong>Use the right stopping criterion:</strong>
        <ul>
        <li>In probability: Stop when P(close) is high enough</li>
        <li>Almost sure: Stop when actual changes are small</li>
        <li>Dist. only: Can't use simple stopping criterion</li>
        </ul>
</li>
</ol>
    
</div>


<h5>🎓 Key Theorems Using Each Type</h5>

<ul>
<li><strong>Central Limit Theorem:</strong> Convergence in distribution</li>
<li><strong>Weak Law of Large Numbers:</strong> Convergence in probability</li>
<li><strong>Strong Law of Large Numbers:</strong> Almost sure convergence</li>
<li><strong>Slutsky's Theorem:</strong> Uses convergence in probability</li>
<li><strong>Continuous Mapping Theorem:</strong> Preserves type of convergence</li>
</ul>

<p style="background: #fff8e1; padding: 15px; border-radius: 8px; margin: 15px 0;">
<strong>Remember:</strong> Understanding convergence types helps you:<br>
- Choose appropriate algorithms<br>
- Set correct stopping criteria<br>
- Interpret your results properly<br>
- Prove theoretical guarantees<br>
- Debug when things go wrong
</p>

## Law of Large Numbers (LLN)

**SCENARIO 1.** Imagine you own a **casino**. You have a simple game:

- Player pays $1 to play
- Roll a fair die (6 sides)
- If it lands on 6: player wins $5 (you lose $4)
- If it lands on 1-5: player wins nothing (you keep $1)

On average, you expect to make **0.17 per game** (let's verify: $-4 \times 1/6 + 1 \times 5/6 = 1/6 \approx 0.17$).

But there's uncertainty. 

Consider what happens:
- After 1 game: You might be up $1 or down $4 (wild swings)
- After 10 games: Still quite variable
- After 100 games: What do you expect?
- After 10,000 games: What about now?

> Should you be worried about going bankrupt on a lucky day for players?
</br>

<div class="alert-exercise">
<h5> QUESTION:</h5> 

Simulate the casino scenario:

1. Calculate the theoretical profit of the casino
2. Simulate up to 10000 games (rolling a fair dice):
    - for each game, calculate the casino profit
    - calculate cumulative sum of profits
3. Calculate running average

```
def simulate_casino_scenario(n_games:int=10000, cost_to_play:float=1.0, payout_on_six:float=5.0) -> tuple[float, np.array]:
    """
    Simulates casino scenario. Calculates the theoretical expected profit of the casino. Then "plays" n_games and calculates the running average of profits.

    Args:
        n_games (int, optional): Number of games (trials) to consider in simulations. Defaults to 10000.
        cost_to_play (float, optional): Cost a player pays to play. Defaults to 1.0.
        payout_on_six (float, optional): Gain in case of winning (die rolls 6). Defaults to 5.0.

    Returns:
        tuple[float, np.array]: expected casino profit and running average of profits.
    """
```

4. Display the theoretical value and empirical values after 50, 500, 5000, and 10000 games.
5. Visualise the result: casino profit as a function of number of games

*Hint*: to calculate a cumulative sum, you can use [`np.cumsum()`](https://numpy.org/doc/2.3/reference/generated/numpy.cumsum.html).

</div>

In [ ]:
# ANSWER
def simulate_casino_scenario(n_games:int=10000, cost_to_play:float=1.0, payout_on_six:float=5.0) -> tuple[float, np.array]:
    """
    Simulates casino scenario. Calculates the theoretical expected profit of the casino. Then "plays" n_games and calculates the running average of profits.

    Args:
        n_games (int, optional): Number of games (trials) to consider in simulations. Defaults to 10000.
        cost_to_play (float, optional): Cost a player pays to play. Defaults to 1.0.
        payout_on_six (float, optional): Gain in case of winning (die rolls 6). Defaults to 5.0.

    Returns:
        tuple[float, np.array]: expected casino profit and running average of profits.
    """
    np.random.seed(42)
    
    pass

In [ ]:
# ANSWER
# display


In [ ]:
# ANSWER
# visualisation


In [ ]:
# visualisation
running_avg, expected, fig = casino_simulation_intro(py=True)

**Key insight**:
- Each individual game is RANDOM and UNPREDICTABLE
- But the AVERAGE over many games becomes PREDICTABLE
- The more games you play, the closer you get to the expected value
- This is why casinos always make money in the long run

**SCENARIO 2.** Let's consider another scenario. This time we are talking about **election poll**. 

You're conducting a poll before an election. 

The Truth (unknown to pollsters):
<ul>
<li>52% of the population supports Candidate A</li>
<li>48% supports Candidate B</li>
</ul>

Your Task: Estimate the support by randomly surveying people

> How many people do you need to survey to get a reliable estimate?
</br>

10 people? 50 people? 100 people? 1000 people?

<div class="alert-exercise">
<h5> QUESTION:</h5> 

Simulate the election poll scenario:

1. Calculate the theoretical profit of the casino
2. Simulate up to 5000 participants (`max_surveys`): 
- Here, we consider that the choice is only between 2 candidates: candidate A and candidate B. So, we can use binomial distribution to model responses
- Calculate cumulative support of candidate A   
3. Calculate running average and errors (difference between the estimate and the true value)

```
def simulate_election_poll_scenario(true_support:float= 0.52, max_surveys:int=5000) -> tuple[np.array, np.array]:
    """
    Simulate the Election Poll Scenario. Two candidates are competing. 
    That's why binomial distribution with p=true_support is used. 
    
    Calculates running average and errors (difference of the estimates with true value).

    Args:
        true_support (float, optional): True support of the first candidate (or Candidate A). Defaults to 0.52.
        max_surveys (int, optional): Max number of participants of the survey. Defaults to 5000.

    Returns:
        running_estimate (np.array): Running average (estimate) of the support of candidate A
        errors (np.array): Difference between the estimates and the true value
    """
```

4. Display the values for the following poll sizes: 10, 50, 100, 500, 1000, 5000.
5. Visualise the results

</div>

In [ ]:
# ANSWER
def simulate_election_poll_scenario(true_support:float= 0.52, max_surveys:int=5000) -> tuple[np.array, np.array]:
    """
    Simulate the Election Poll Scenario. Two candidates are competing. 
    That's why binomial distribution with p=true_support is used. 
    
    Calculates running average and errors (difference of the estimates with true value).

    Args:
        true_support (float, optional): True support of the first candidate (or Candidate A). Defaults to 0.52.
        max_surveys (int, optional): Max number of participants of the survey. Defaults to 5000.

    Returns:
        running_estimate (np.array): Running average (estimate) of the support of candidate A
        errors (np.array): Difference between the estimates and the true value
    """
    np.random.seed(42)
    
    pass

In [ ]:
# ANSWER
    

In [ ]:
# ANSWER
# visualisation


In [ ]:
# demo
polling_simulation()

**Key insight:**
- Small polls ($n=50$): Estimates vary widely from poll to poll
- Large polls ($n=1000$): Estimates cluster tightly around true value
- As sample size increases, estimate converges to true population value
- This is why professional polls typically survey 1000-1500 people. You don't need to survey everyone - just a large enough sample.

**Common Thread:**
<ul>
<li>Each individual observation is RANDOM and UNPREDICTABLE</li>
<li>But the AVERAGE of many observations becomes PREDICTABLE</li>
<li>The more observations, the closer we get to the "true value"</li>
<li>This convergence happens REGARDLESS of the distribution!</li>
</ul>

This is the **Law of Large Numbers** (LLN).

<p style="background: #fff; padding: 15px; border-radius: 8px; border-left: 5px solid #ff9800; margin: 15px 0;">
<strong>Informal Statement:</strong><br>
When you repeat a random experiment many times and take the average, that average will get closer and closer to the expected value.

In [ ]:
# insights 
bridge_to_formal_definition()

<div class="alert alert-success">
<h4>Definition: The Law of Large Numbers</h4>

Setup: Let $X_1, X_2, X_3, ...$ be independent, identically distributed (i.i.d.) random variables with:</p>
- Expected value: $E[X_i] = \mu$
- Variance: $Var(X_i) = \sigma^2 < \infty$ 

The sample mean is defined as: $$\bar{X_n} = (X_1 + X_2 + ... + X_n) / n$$

<h5>Weak Law of Large Numbers (WLLN):</h5>

$$\bar{X_n} \xrightarrow[n\rightarrow\infty]{p} \mu \text{ (converges in probability)}$$

*Formally*: For any $\varepsilon > 0$,
$$P(|\bar{X_n} - \mu| > \varepsilon) \rightarrow 0\text{ as } n \rightarrow \infty$$

*In words*: The probability that the sample mean is "far" from $\mu$ goes to zero.

<h5>Strong Law of Large Numbers (SLLN):</h5>

$$\bar{X_n} \xrightarrow[n\rightarrow\infty]{a.s.} \mu \text{ (converges almost surely)}$$

*Formally:*

$$P(\bar{X_n} \rightarrow \mu \text{ as } n \rightarrow \infty) = 1$$

*In words*: If you actually run the experiment, the sequence $\bar{X_1}, \bar{X_2}, \bar{X_3}, ...$ will converge to $\mu$ (with probability 1).


**Intuitive Interpretation:**

1. WLLN: "For large $n$, $\bar{X_n}$ is *probably* close to $\mu$"
2. SLLN: "If you run the experiment, $\bar{X_n}$ *will* converge to $\mu$ (almost certainly)"


**Key Requirements:**

- Independence: Samples must be independent
- Identical distribution: All from the same distribution
- Finite mean: $E[X]$ must exist
- Finite variance: $Var(X) < \infty$ (for WLLN; SLLN only needs finite mean)

</div>


<div class="alert alert-idea">
<h4>💡Key Insight</h4>

In every case, averaging many independent random observations gives us a reliable estimate of the expected value.

Why This Matters:
<ul>
<li><strong>Statistics:</strong> Justifies using sample means to estimate population means</li>
<li><strong>Machine Learning:</strong> Explains why Monte Carlo methods work</li>
<li><strong>Probability Theory:</strong> Foundation for Central Limit Theorem and inference</li>
<li><strong>Real World:</strong> Why casinos make money, why polls work, why averaging reduces noise</li>
</ul>

**What LLN Does NOT Tell Us**

<p>The Law of Large Numbers guarantees convergence but is <strong>silent on the rate</strong>.</p>

Questions LLN Cannot Answer:
<ul>
<li>How many samples do we need to be within ε of μ?</li>
<li>How much does the error decrease when we double the sample size?</li>
<li>What's the distribution of X̄ₙ - μ for finite n?</li>
</ul>

For these answers, we need the Central Limit Theorem.

</div>

## Central Limit Theorem (CLT)

<div class="alert-exercise">
<h5> QUESTION:</h5> 

We have several probability distributions with very different shapes:

- $Uniform(0, 1)$: Flat, every value equally likely
- $Exponential(\lambda=1)$: Heavily right-skewed
- $Normal(\mu=0, \sigma^2=1)$: Standard normal with bell-shaped PDF
- $Binomial(n=10, p=0.5)$: Discrete, symmetric but not continuous
- $Poisson(\lambda=5)$: Discrete

The Experiment:

1. Pick a distribution (start with any one you like)
2. Draw $n$ samples from it (e.g., $n=5$, $n=30$, $n=100$)
3. Calculate the **mean** of those $n$ samples
4. Repeat step 2-3 many times (e.g., 1000 times)
5. Plot a histogram of all those sample means
6. Try different values of $n$ and different distributions

Questions to Investigate:

1. What shape does the histogram of sample means have?
2. Does this shape depend on which distribution you started with?
3. How does the shape change as you increase $n$ (sample size)?
4. How does the spread (width) of the histogram change as $n$ increases?
5. Where is the center of the sample means distribution?

</div>

*Hints:*

You'll need these tools:

```
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt

```

1. Step 1: Create a distribution object

```
# Example: Exponential distribution
dist = stats.expon(scale=1)

# You can also try:
# dist = stats.uniform(loc=0, scale=1)
# dist = stats.binom(n=10, p=0.5)
```

2. Step 2: Generate ONE sample and compute its mean

```
sample_size = 30  # Try 5, 10, 30, 100
one_sample = dist.rvs(size=sample_size)
sample_mean = np.mean(one_sample)
print(f"One sample mean: {sample_mean}")
```

3. Step 3: Repeat many times to collect sample means

```
n_repetitions = 1000
sample_means = []

for i in range(n_repetitions):
    sample = dist.rvs(size=sample_size)
    sample_means.append(np.mean(sample))

# Convert to numpy array
sample_means = np.array(sample_means)
```

4. Step 4: Visualize

```
plt.figure(figsize=(10, 6))
plt.hist(sample_means, bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Sample Mean')
plt.ylabel('Frequency')
plt.title(f'Distribution of Sample Means (n={sample_size})')
plt.grid(True, alpha=0.3)
plt.show()
```

You can create a 2×2 grid comparing different sample sizes ($n=5$, $n=30$, $n=100$, $n=1000$)


In [ ]:
# ANSWER
def clt_solution(n_samples=1000):
    """
    Simplified solution showing CLT for multiple distributions and sample sizes.

    Args:
        n_samples (int, optional): Number of sample means to generate. Defaults to 1000.

    Returns:
        fig: matplotlib figure
    """
    
    # Set random seed for reproducibility
    np.random.seed(42)
    
    pass

In [ ]:
# Run the solution
clt_solution(n_samples=10)
clt_solution(n_samples=100)
clt_solution(n_samples=1000)

**KEY OBSERVATIONS:**

1. SHAPE BECOMES NORMAL
    -   No matter which distribution you started with
    -   The histogram of sample means looks bell-shaped (normal)
    
2. SAMPLE SIZE MATTERS
    -  Left to right: as n increases, the shape becomes MORE normal
    -  With $n=5$: still somewhat irregular
    -  With $n=30$ or more: very close to normal shape
    
3. SPREAD DECREASES
    -  Left to right: the histogram gets NARROWER
    - This happens in a predictable way: $SE = \sigma/\sqrt{n}$
    - Larger n → smaller standard error → more precise estimates
    
4. CENTER STAYS THE SAME
    - The center (mean) of $\text{sample means} \approx \text{population mean}$
    - This is true regardless of $n$
    - Sample mean is an unbiased estimator!
    
5. THE PATTERN IS UNIVERSAL
    - Works for continuous distributions (Uniform, Exponential, Normal)
    - Works for discrete distributions (Binomial, Poisson)
    - Works for symmetric distributions (Binomial, Uniform)
    - Works for skewed distributions (Exponential)
    

<div class="alert alert-summary">
<h4>scipy.stats Cheat Sheet</h4>

<h5>1️⃣ Creating a Distribution Object</h5>

```
from scipy import stats

# Continuous distributions
dist = stats.norm(loc=0, scale=1)           # Normal
dist = stats.uniform(loc=0, scale=1)        # Uniform
dist = stats.expon(scale=1)                 # Exponential
dist = stats.gamma(a=2, scale=2)            # Gamma
dist = stats.beta(a=2, b=5)                 # Beta

# Discrete distributions
dist = stats.binom(n=10, p=0.5)             # Binomial
dist = stats.poisson(mu=5)                  # Poisson

```

<h5>2️⃣ Generating Random Samples</h5>

```
# Generate n random samples
samples = dist.rvs(size=100)

# With random seed for reproducibility
samples = dist.rvs(size=100, random_state=42)

# Generate multiple samples (for CLT demo)
sample_means = [dist.rvs(size=30).mean() for _ in range(1000)]

```

<h5>3️⃣ Getting Distribution Parameters</h5>

```
mean = dist.mean()                          # Population mean
std = dist.std()                            # Population std dev
var = dist.var()                            # Population variance
median = dist.median()                      # Median

# All moments at once
mean, var, skew, kurt = dist.stats(moments='mvsk')
```


<h5>4️⃣ Probability Functions</h5>

```
# PDF (continuous) or PMF (discrete)
prob = dist.pdf(x)                          # For continuous
prob = dist.pmf(x)                          # For discrete

# CDF
cumulative = dist.cdf(x)                    # P(X ≤ x)

# Survival function
survival = dist.sf(x)                       # P(X > x) = 1 - CDF(x)

# Percent point function (inverse CDF)
quantile = dist.ppf(0.95)                   # 95th percentile

```

<h5>5️⃣ CLT Demonstration Template</h5>

```
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt

# 1. Create distribution
dist = stats.expon(scale=2)

# 2. Generate sample means
n_samples = 1000
sample_size = 30
sample_means = [dist.rvs(size=sample_size).mean() 
                for _ in range(n_samples)]

# 3. Get theoretical values
mu = dist.mean()
sigma = dist.std()
se = sigma / np.sqrt(sample_size)

# 4. Plot
plt.hist(sample_means, bins=50, density=True, alpha=0.7)
x = np.linspace(min(sample_means), max(sample_means), 100)
plt.plot(x, stats.norm.pdf(x, mu, se), 'r-', linewidth=2)
plt.title(f'CLT: Sample Means ~ N({mu:.2f}, {se:.4f}²)')
plt.show()

# 5. Verify normality
from scipy.stats import shapiro
stat, p_value = shapiro(sample_means)
print(f'Shapiro-Wilk test: p-value = {p_value:.4f}')

```

<h5>6️⃣ Useful Tips</h5>
<ul>
<li><strong>Freeze distribution:</strong> Create once, use many times for efficiency</li>
<li><strong>Random state:</strong> Always set random_state for reproducibility</li>
<li><strong>Vectorization:</strong> Pass arrays to .pdf(), .cdf(), etc. for speed</li>
<li><strong>Documentation:</strong> Use <code>help(stats.norm)</code> or <code>stats.norm?</code> in Jupyter</li>
<li><strong>List all distributions:</strong> <code>[d for d in dir(stats) if isinstance(getattr(stats, d), type)]</code></li>
</ul>

<h5>7️⃣ Complete Distribution List</h5>
<p><strong>Continuous:</strong> norm, uniform, expon, gamma, beta, chi2, t, f, lognorm, weibull_min, pareto, cauchy, laplace, logistic, gumbel_r, rayleigh, and 80+ more!</p>
<p><strong>Discrete:</strong> binom, poisson, geom, nbinom, hypergeom, zipf, and 10+ more!</p>

<p><strong>Documentation:</strong> <a href="https://docs.scipy.org/doc/scipy/reference/stats.html" target="_blank">scipy.stats official docs</a></p>
</div>


In [ ]:
# demo
show_transformation()

We've just discovered the Central Limit Theorem empirically.

<div class="alert alert-success">
<h4>Definition: Central Limit Theorem (CLT)</h4>

<p><strong>The Central Limit Theorem states:</strong></p>

When you take random samples of size $n$ from ANY distribution (with finite mean $\mu$ and variance $\sigma^2$), and calculate their means, those sample means will follow an approximately **normal distribution** with mean $\mu$ and standard deviation $\sigma/\sqrt{n}$.


<p><strong>In mathematical notation:</strong></p>


Let $X_1, X_2, ..., X_n$ be i.i.d. with mean $\mu<\infty$ and std $\sigma<\infty$. Then:
$$\bar{X_n} \sim N(\mu, \sigma^2/n) \text{ as } n \rightarrow \infty$$

Another form:

Let $S_n = \sum_{i}^n X_i$. Then, using the standardisation procedure:
$$\frac{S_n-n\mu}{\sigma \sqrt{n}} \xrightarrow[n\rightarrow + \infty]{\mathcal{d}} Y \sim {N}(0,1)$$

**Key properties:**

- Universality: Works for ANY starting distribution
- Normality: Result is always approximately normal
- Predictability: We can predict the standard error: $SE = \sigma/\sqrt{n}$
- Practicality: Explains why normal distribution appears everywhere in nature and ML</li>

</div>

<p><strong>Why this matters for Machine Learning:</strong></p>
<ul>
<li><strong>Monte Carlo methods:</strong> We can quantify uncertainty in our estimates</li>
<li><strong>Gradient descent:</strong> Understand noise in mini-batch gradients</li>
<li><strong>Statistical inference:</strong> Build confidence intervals and hypothesis tests</li>
<li><strong>Model evaluation:</strong> Understand variability in performance metrics</li>
<li><strong>A/B testing:</strong> Determine required sample sizes</li>
</ul>

<p><strong>The √n rule:</strong> This is THE fundamental rate in statistics and ML</p>
<ul>
<li>To halve your error, you need 4× more samples</li>
<li>To get 10× better accuracy, you need 100× more data</li>
<li>This explains why "big data" is so important</li>
</ul>

> Why finite variance is crucial? </br>

Consider the formula $\frac{S_n-n\mu}{\sigma \sqrt{n}} \xrightarrow[n\rightarrow + \infty]{\mathcal{d}} Y \sim {N}(0,1)$. Notice that $\sigma$ appears in the denominator. If $\sigma^2 = \infty$, this formula becomes meaningless:
- We're dividing by infinity
- The rate of convergence ($1/\sqrt{n}$) depends on $\sigma$ being finite
- The limiting distribution's variance comes from $\sigma^2/n$

*Intuition*: Variance measures "typical deviation from the mean." If variance is infinite, there's no "typical" scale - extreme values are so common that averaging doesn't help.

<div class="alert alert-danger">
<h4>⚠️ Common Mistake: When CLT Fails</h4>

<p><strong>CLT requires finite variance!</strong> If σ² = ∞, CLT does not apply.</p>

<p><strong>Example: Cauchy Distribution</strong></p>
<ul>
<li>Has undefined mean and infinite variance</li>
<li>Sample means do NOT converge to normal</li>
<li>Sample means follow the SAME Cauchy distribution (wild!)</li>
</ul>

<p><strong>ML Implications:</strong></p>
<ul>
<li><strong>Heavy-tailed losses:</strong> Some loss functions (e.g., with outliers) might have infinite variance</li>
<li><strong>Learning rates:</strong> If gradients have infinite variance, standard convergence theory breaks</li>
<li><strong>Robust statistics:</strong> Use median instead of mean for heavy-tailed data</li>
</ul>

<p><strong>Rule of thumb:</strong> CLT works well when n ≥ 30 for most distributions. For heavily skewed distributions, might need n ≥ 100.</p>
</div>


In [ ]:
# demo Cauchy distribution
demo_clt_cauchy()

The Cauchy distribution is the canonical example of infinite variance. its PDF: $f_X(x) = \frac{1}{\pi(1 + x^2)}$.

Properties:
- Mean: undefined (integral doesn't converge)
- Variance: infinite ($\sigma^2 = \infty$)
- Heavy tails: $P(|X| > x) \sim 1/x$ (much heavier than normal)

The critical difference between Normal and Cauchy distribution is in the TAILS, which are hard to see in a histogram, especially in a linear scale (that's why in the demo above a log-scale is used).

1. Cauchy has such heavy tails that extreme values dominate
2. One extremely large value can completely change the mean
3. As $n$ grows, you're increasingly likely to hit an extreme value
4. These extremes keep the variance infinite
5. Averaging doesn't reduce variability

*Mathematical property* (stability under averaging):
The Cauchy distribution is "stable" - the sum (or mean) of Cauchy random variables is still Cauchy. This is extremely unusual.

> What is the difference between LLN and CLT? </br>

**Law of Large Numbers (LLN)**:

LLN tells you WHERE the sample mean goes (it converges to μ), but not HOW FAST or what the distribution looks like.

In practical terms: *"Your estimate will be close to the truth with enough data"*

**Central Limit Theorem (CLT)**:

CLT tells you HOW FAST the sample mean converges (at rate 1/√n) and WHAT SHAPE the distribution has (approximately normal), allowing you to quantify uncertainty.

In practical terms: *"Here's how close, with what probability, and how much data you need"*

## Problem-Solving Strategy

<div class="alert alert-idea">
<h4>💡 Problem-Solving Strategy: How to Apply These Concepts</h4>
<span class="idea-type">General Framework</span>

<div class="idea-steps">
<h5>Step-by-Step Approach:</h5>
<ol>
<li><strong>Identify the estimator:</strong> What are you averaging? (Sample mean, gradient, Monte Carlo estimate, etc.)</li>

<li><strong>Check LLN applicability:</strong>
   <ul>
   <li>Are samples independent?</li>
   <li>Identically distributed?</li>
   <li>Finite mean?</li>
   </ul>
   If YES → Estimator converges to true value</li>

<li><strong>Check CLT applicability:</strong>
   <ul>
   <li>All LLN conditions PLUS</li>
   <li>Finite variance?</li>
   </ul>
   If YES → Can quantify uncertainty and convergence rate</li>

<li><strong>Apply the √n rule:</strong>
   <ul>
   <li>Standard error: SE = σ/√n</li>
   <li>95% CI width: ≈ 4 × SE = 4σ/√n</li>
   <li>To improve accuracy by factor k: need k² times more samples</li>
   </ul>
</li>

<li><strong>Estimate required sample size:</strong>
   <ul>
   <li>Desired accuracy: ε</li>
   <li>For 95% confidence: need SE ≈ ε/2</li>
   <li>Solve: σ/√n = ε/2 → n = (2σ/ε)²</li>
   </ul>
</li>
</ol>
</div>

<p><strong>Common ML Applications:</strong></p>
<ul>
<li><strong>Monte Carlo methods:</strong> Use this framework to determine number of samples</li>
<li><strong>Gradient estimation:</strong> Understand mini-batch size vs noise trade-off</li>
<li><strong>Bootstrap:</strong> Determine number of bootstrap iterations</li>
<li><strong>A/B testing:</strong> Calculate required sample size for desired statistical power (coming in Week 8!)</li>
</ul>
</div>


<div class="alert alert-exercise">
<h4>Integration Exercise: Monte-Carlo Method for π Estimation</h4>

*Scenario*: You're implementing a Monte Carlo method to estimate π using the classic quarter-circle method.

*Method*: Generate random points $(x,y)$ in $[0,1]\times[0,1]$. Let $X_1, X_2, ... , X_n$ and $Y_1, Y_2, ..., Y_n$ be independent variables with distribution $\mathcal{U}[0; 1]$.
Count how many fall inside the quarter circle ($x^2 + y^2 \leq 1$). 
The ratio estimates $\pi/4$.</p>

Tasks:

1. Let $P_n = \frac{4}{n}\sum_{i=1}^{n}Z_i$ where $Z_i = \mathbf{1}_{X^2_i + Y^2_i\leq 1} \sim Bernoulli(p=\pi/4)$. Show that $P_n$ converges almost surely to $\pi$
2. Let $\alpha > 0$. Using Chebyshev's inequality, determine $n_\alpha$ such that for all $n$
greater than $n_\alpha$, $\mathbb{P}(|P_n - \pi| > \alpha) \leq 0.05$
3. Repeat the previous question using CLT
4. Use the implementation of the estimator
5. Run it with different sample sizes: 100, 1000, 10000, 100000
6. For each sample size, repeat 1000 times to see the distribution
7. Verify that:
   - Estimates converge to π (Law of Large Numbers)
   - Distribution of estimates is approximately normal (Central Limit Theorem)
   - Standard error decreases as 1/√n (CLT rate)
8. Determine: How many samples needed to estimate π within ±0.01 with 95% confidence?</li>
</ol>

</div>

In [ ]:
def estimate_pi(n_samples):
    """Single estimate of π using Monte Carlo"""
    x = np.random.uniform(0, 1, n_samples)
    y = np.random.uniform(0, 1, n_samples)
    inside_circle = (x**2 + y**2) <= 1
    pi_estimate = 4 * np.mean(inside_circle)
    return pi_estimate

<details>
<summary>Reveal Solution</summary>

1. Almost Sure Convergence

Let's denote $R^2_i = X^2_i + Y^2_i$.

*Independence*

**Lemma:** Let $n$ be a natural number, $X_1$, $X_2$, ... , $X_n$ be independent random variables and $k$ be an integer between $1$ and $n$. Let $\varphi: \mathbb{R}^k \rightarrow \mathbb{R}$ and $\psi :  \mathbb{R}^{n-k} \rightarrow \mathbb{R}$ be measurable mappings. Then the variables $Y = \varphi(X_1, ... , X_k)$ and $Z = \psi(X_{k+1}, ... , X_n)$ are independent. 

Consequently, the variables $R^2_i = X^2_i + Y^2_i$ are independent and $Z_i = \mathbf{1}_{R^2_i\leq 1}$ too. 

*Identical distribution*

The variables $R_i$ come from the same distribution. The condition $R_i^2 \leq 1$ is the same for all $Z_i$. Then $Z_i$ come from the same distribution.

According to the problem statement, $Z_i = \mathbf{1}_{R^2_i\leq 1} \sim \mathcal{B}(\pi/4)$. The expectation of $Z_i$ is therefore $m=\pi/4$.

According to the **strong law of large numbers**: $$\lim_{n\rightarrow +\infty}\overline{X}_n = m$$

In our case, 

$$\lim_{n\rightarrow +\infty}\overline{Z}_n =\frac{1}{n}\sum_{i=1}^{n}Z_i = m = \pi/4$$

The expression in question on the left is multiplied by 4 compared to that of the law, so: 

$$\frac{4}{n}\sum_{i=1}^{n}Z_i = 4m = 4\cdot (\pi/4) = \pi$$

Hence, the convergence almost surely.

2. Finding $n_\alpha$ using Chebyshev's inequality

According to **Chebyshev's inequality**: $\forall \alpha \in \mathbb{R}, \alpha>0$:

$$\mathbb{P}(|X - m| \geq \alpha) \leq \frac{\sigma^2}{\alpha^2}$$

In our case, $Z_i \sim \mathcal{B}(\pi/4)$, $P_n = \frac{4}{n}\sum_{i=1}^{n}Z_i$ which converges almost surely to $\pi$, its expectation $m = \pi$ and variance:

$$\sigma^2 = \sum_{i=1}^{n} \frac{4}{n}Var(Z_i) =\bigg(\frac{4}{n}\bigg)^2 \sum_{i=1}^{n} Var(Z_i) = \bigg(\frac{4}{n}\bigg)^2 np(1-p) = \frac{16}{n^2}\frac{n\pi}{4}(1-\pi/4) = \frac{4\pi}{n}(1-\pi/4)$$

Then, 

$$\mathbb{P}(|X - m| \geq \alpha) = \mathbb{P}(|P_n - m| \geq \alpha) = \mathbb{P}(|P_n - \pi| \geq \alpha) \leq \frac{\frac{4\pi}{n}(1-\pi/4)}{\alpha^2}$$

Let's take up the right part. According to the problem statement it must be less than 0.05:
$$\frac{\frac{4\pi}{n}(1-\pi/4)}{\alpha^2} \leq 0.05$$

$$\frac{4\pi}{n}(1-\pi/4) \leq 0.05\alpha^2$$

$$\frac{4\pi(1-\pi/4)}{0.05\alpha^2} \leq n$$

3. Finding $n_\alpha$ using CLT

According to the CLT:

$$\frac{S_n-nm}{\sigma \sqrt{n}} \xrightarrow[n\rightarrow + \infty]{\mathcal{d}} Y \sim\mathcal{N}(0,1)$$
where $S_n=\sum_{i=1}^{n}X_i$.

In our case, (the standardization) 

$$U = \frac{X-m}{\sigma} = \frac{P_n - \pi}{\sqrt{\frac{4\pi}{n}(1-\pi/4)}} \xrightarrow[n\rightarrow + \infty]{\mathcal{L}} \mathcal{N}(0,1)$$

Then:

$$\mathbb{P}(|P_n - \pi| \geq \alpha) = \mathbb{P}\left(|U| \geq \frac{\alpha}{\sqrt{\frac{4\pi}{n}(1-\pi/4)}}\right) \leq 0.05$$
Note that the expression under the probability contains the absolute value $|U|$. 

Recall the symmetry of the normal distribution:

<center>
<img src="img/normal-absolute-value.png" alt="Symmetry of Normal distribution" width="400px">
</center>

That's why, we need to consider the value $0.025$. Using the `scipy.stats.norm.ppf` method, we obtain $u=1.96$.

Therefore:

$$\frac{\alpha}{\sqrt{\frac{4\pi}{n}(1-\pi/4)}} \geq 1.96$$

Then, 

$$\frac{\alpha^2}{\frac{4\pi}{n}(1-\pi/4)} \geq 1.96^2$$

$$\frac{n\alpha^2}{4\pi(1-\pi/4)} \geq 1.96^2$$

$$n \geq \frac{1.96^2 \cdot 4\pi(1-\pi/4)}{\alpha^2}$$

Let's introduce the effective base variance that is independent of $n$ as:
$$\sigma_0^2 ​ = \frac{Var(4\sum_{i=1}^n ​Z_i​)}{n} = \frac{16\cdot \sum_{i=1}^n Var(Z_i​)}{n} = \frac{16\cdot n Var(Z_i​)}{n} = 16 Var(Z_i​) = 16 \pi/4 (1 − \pi/4) = 4\pi(1-\pi/4)$$

In this case, the inequality becomes:

$$n \geq \frac{1.96^2 \cdot \sigma_0^2}{\alpha^2}$$

Let's compare the obtained results:

|error| Chebychev's inequality | CLT |
|---:|:---:|:---:|
|general case| $$n \geq \frac{4\pi(1-\pi/4)}{0.05\alpha^2}$$ | $$n \geq \frac{1.96^2 \cdot 4\pi(1-\pi/4)}{\alpha^2}$$ |
|$\alpha = 0.01$ | $$n \geq \frac{4\pi(1-\pi/4)}{0.05\cdot 0.01^2}\approx 539,354$$ | $$n \geq \frac{1.96^2 \cdot 4\pi(1-\pi/4)}{0.01^2}\approx 103,599$$ | 

We note that there is a factor of 0.05 with the $n_\alpha$ calculated in the previous question. The central limit theorem gives a finer and presumably more accurate value.
We just need to make sure it is high enough so that we can consider the approximation given by the CLT valid.

</details>


In [ ]:
# get the value of the CDF of standard normal distribution for u=0.025
U = stats.norm(loc=0, scale=1)
print(U.ppf(0.025))

In [ ]:
common_term = 4*np.pi*(1-np.pi/4)
alpha = 0.01

chebyshev_est = common_term / (0.05 * alpha**2)
clt_est = 1.96**2 * common_term / alpha**2

print(f"Using Chebyshev's inequality: {chebyshev_est}")
print(f"Using CLT: {clt_est}")

<div class="alert alert-success">
<h4>Formula: Required Sample Size for Given Precision</h4>

Let $X_i$ be i.i.d. with mean $\mu$.

To estimate a mean $\mu$ with a margin of error $\pm \varepsilon$, confidence level $(1 - \alpha)$ (typically 95%, so $\alpha=0.05$), and population standard deviation $\sigma$, a required sample size is given by:

$$n = \bigg(\frac{z_{\alpha/2}\cdot \sigma}{\varepsilon}\bigg)^2$$

where $z_{\alpha/2}$ is the critical value from standard normal distribution $N(0, 1)$.

Some typical critical values:
- For 95% confidence: $z_{0.025} = 1.96 \approx 2$
- For 99% confidence: $z_{0.005} = 2.576 \approx 2.6$
- For 90% confidence: $z_{0.05} = 1.645 \approx 1.6$

</div>

In [ ]:
# ANSWER
sample_sizes = [100, 1000, 10000, 100000, 1000000]
n_trials = 1000

In [ ]:
def pi_convergence_analysis(sample_sizes, n_trials=1000):    
    fig, axes = plt.subplots(2, 3, figsize=(16, 10))
    true_pi = np.pi
    
    for idx, n in enumerate(sample_sizes):
        row, col = idx // 3, idx % 3
        ax = axes[row, col]
        
        # Generate multiple estimates
        estimates = [estimate_pi(n) for _ in range(n_trials)]
        estimates = np.array(estimates)
        
        # Plot histogram
        ax.hist(estimates, bins=50, density=True, alpha=0.7, 
                color='skyblue', edgecolor='black')
        
        # Overlay theoretical normal
        mean_est = np.mean(estimates)
        std_est = np.std(estimates)
        x = np.linspace(estimates.min(), estimates.max(), 1000)
        y = stats.norm.pdf(x, mean_est, std_est)
        ax.plot(x, y, 'r-', linewidth=2, label=f'N({mean_est:.3f}, {std_est:.3f}²)')
        
        # Mark true value
        ax.axvline(true_pi, color='green', linestyle='--', linewidth=2, 
                   label=f'True π = {true_pi:.4f}')
        
        ax.set_title(f'n = {n:,}\nMean: {mean_est:.4f}, SE: {std_est:.4f}', 
                     fontsize=11, fontweight='bold')
        ax.set_xlabel('π Estimate')
        ax.set_ylabel('Density')
        ax.legend(fontsize=8)
        ax.grid(True, alpha=0.3)
        
        # Print statistics
        error = abs(mean_est - true_pi)
        within_01 = np.mean(np.abs(estimates - true_pi) < 0.01) * 100
        print(f"n = {n:6d}: Mean = {mean_est:.5f}, SE = {std_est:.5f}, "
              f"Error = {error:.5f}, Within ±0.01: {within_01:.1f}%")
    
    plt.suptitle('Monte Carlo Estimation of π: Convergence Analysis', 
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.close()
    return fig

In [ ]:
pi_convergence_analysis(sample_sizes=sample_sizes, n_trials=1000)

In [ ]:
def analyse_se(sample_sizes, n_trials=1000):
    # Plot SE vs n (log-log scale)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Collect observed standard errors
    std_errors = []
    for n in sample_sizes:
        estimates = [estimate_pi(n) for _ in range(n_trials)]
        # as each estimate is a sample mean according to Monte-Carlo method,
        # the std of these sample means is, by def. the standard error
        std_errors.append(np.std(estimates))
    
    # Plot 1: SE vs n
    ax1.plot(sample_sizes, std_errors, 'bo-', linewidth=2, markersize=8, label='Observed SE')
    
    # estimate sigma from one sample size (recover sigma from observed SE for sample size n0)
    sigma_estimate = std_errors[0] * np.sqrt(sample_sizes[0])
    # Theoretical line (1/√n behavior)
    theoretical_se = sigma_estimate / np.sqrt(np.array(sample_sizes))
    ax1.plot(sample_sizes, theoretical_se, 'r--', linewidth=2, label='Theoretical: ' + r"$\propto$" + '1/√n')
    
    ax1.set_xlabel('Sample Size (n)', fontsize=12)
    ax1.set_ylabel('Standard Error', fontsize=12)
    ax1.set_title('Standard Error vs Sample Size', fontsize=13, fontweight='bold')
    ax1.set_xscale('log')
    ax1.set_yscale('log')
    ax1.legend(fontsize=11)
    ax1.grid(True, alpha=0.3, which='both')
    
    # Plot 2: Required samples for different error tolerances
    target_errors = np.array([0.1, 0.05, 0.01, 0.005, 0.001])
    conf_level = 0.95
    alpha = 1 - conf_level
    z_alpha_05 = stats.norm.ppf(alpha/2, loc=0, scale=1)
    # Using rule: for 95% CI, need SE ≈ error/2 (1.96 ≈ 2)
    # SE = sigma/√n, so n = (sigma/SE)² = (sigma/(error/2))² = (2sigma/error)²
    
    # Estimate sigma from our data (recover sigma from observed SE for sample size n0)
    sigma_estimate = std_errors[0] * np.sqrt(sample_sizes[0])
    required_samples = (z_alpha_05 * sigma_estimate / target_errors) ** 2
    
    ax2.barh(range(len(target_errors)), required_samples, color='coral', edgecolor='black')
    ax2.set_yticks(range(len(target_errors)))
    ax2.set_yticklabels([f'±{e:.3f}' for e in target_errors])
    ax2.set_xlabel('Required Sample Size', fontsize=12)
    ax2.set_ylabel('Desired Accuracy (95% CI)', fontsize=12)
    ax2.set_title('Samples Needed for Different Accuracy Levels', fontsize=13, fontweight='bold')
    ax2.set_xscale('log')
    ax2.grid(True, alpha=0.3, axis='x')
    
    # Add value labels
    for i, v in enumerate(required_samples):
        ax2.text(v, i, f' {int(v):,}', va='center', fontsize=10, fontweight='bold')
    
    plt.tight_layout()
    plt.close()
    
    return fig

In [ ]:
analyse_se(sample_sizes=sample_sizes, n_trials=1000)

## Return to the Opening Challenge

Let's answer all four questions now.

<div>
<p><strong>Question 1: Will we EVER get exactly 0.7000?</strong></p>
<p><strong>Answer:</strong> Almost certainly NO (probability = 0). But by the <strong>Strong Law of Large Numbers</strong>, 
we'll converge to 0.7 almost surely. That is, P(estimate → 0.7) = 1.</p>

<p style="background: white; padding: 10px; border-radius: 5px; margin: 10px 0;">
<em>LLN tells us: We'll get arbitrarily close, but not exactly equal.</em>
</p>

<hr>

<p><strong>Question 2: How fast does the "jumpiness" decrease?</strong></p>
<p><strong>Answer:</strong> By the <strong>Central Limit Theorem</strong>, the standard error decreases as <strong>1/√n</strong>.</p>


$$SE(estimate) = \sigma / \sqrt{n} \approx 0.458 / \sqrt{n}$$

(where $\sigma = \sqrt{p(1-p)} = \sqrt{0.7 \times 0.3} \approx 0.458$)


<table style="margin: 10px auto; border-collapse: collapse; background: white;">
<tr style="background: #e3f2fd;"><th style="padding: 8px; border: 1px solid #ccc;">Samples (n)</th><th style="padding: 8px; border: 1px solid #ccc;">SE</th><th style="padding: 8px; border: 1px solid #ccc;">95% CI Width</th></tr>
<tr><td style="padding: 8px; border: 1px solid #ccc;">100</td><td style="padding: 8px; border: 1px solid #ccc;">0.046</td><td style="padding: 8px; border: 1px solid #ccc;">±0.090</td></tr>
<tr><td style="padding: 8px; border: 1px solid #ccc;">1,000</td><td style="padding: 8px; border: 1px solid #ccc;">0.014</td><td style="padding: 8px; border: 1px solid #ccc;">±0.028</td></tr>
<tr><td style="padding: 8px; border: 1px solid #ccc;">10,000</td><td style="padding: 8px; border: 1px solid #ccc;">0.0046</td><td style="padding: 8px; border: 1px solid #ccc;">±0.009</td></tr>
<tr><td style="padding: 8px; border: 1px solid #ccc;">100,000</td><td style="padding: 8px; border: 1px solid #ccc;">0.0014</td><td style="padding: 8px; border: 1px solid #ccc;">±0.003</td></tr>
</table>

<hr>

<p><strong>Question 3: Can we guarantee we're within 0.01 of the true value?</strong></p>
<p><strong>Answer:</strong> YES! Using CLT, we can construct confidence intervals.</p>


For 95% confidence with margin of error ±0.01:<br>
$n = (1.96 \times \sigma / 0.01)² = (1.96 \times 0.458 / 0.01)^2 \approx 8,068\text{ samples}$


<p><em>With 8,068 samples, we're 95% confident our estimate is within ±0.01 of 0.7</em></p>

<hr>

<p><strong>Question 4: What if we only have 100 samples (1 second)?</strong></p>
<p><strong>Answer:</strong> Tell your manager the uncertainty:</p>

With 100 samples, our estimate is $\hat{p}_{100} = \frac{1}{100}\sum_{i=1}^100X_i$ where $X_i\sim Bernoulli(p=0.7)$. 

Properties of this estimator:
- Mean $E[\hat{p}_{100}] = 0.7$
- Variance $Var(\hat{p}_{100}) = \frac{p(1-p)}{n} = \frac{0.7(1-0.7)}{100} = \frac{0.21}{100} = 0.0021$
- Standard Error: $SE = \sqrt{0.0021} = 0.0458$

By CLT, for $n=100$:
$$\hat{p}_{100} \sim N(0.7, 0.0458^2)$$

For a 95% confidence interval: $\hat{p}_{100} \pm z_{0.025}\times SE$ where $z_{0.025} = 1.96$ (from Standard Normal CDF).

Margin of error: $z_{0.025}\times SE = 1.96 \times 0.0458 = 0.0898 \approx 0.09$

Therefore, our estimate $0.7 \pm 0.09$ (95\% CI). 

That means the true probability could be anywhere from 0.61 to 0.79. <br>
For a ±0.01 precision, we need 8,000 samples (80 seconds).


<p><strong>Trade-off:</strong> Speed vs Precision - a fundamental constraint in probabilistic systems</p>

</div>

## Common Mistakes

<div class="alert alert-danger">
<h5>⚠️ Common Pitfalls</h5>

<ol>
<li><strong>Assuming fast convergence:</strong> √n is slow! 100× better accuracy needs 10,000× more data</li>
<li><strong>Ignoring independence:</strong> LLN/CLT require independent samples</li>
<li><strong>Infinite variance:</strong> CLT fails for heavy-tailed distributions (check your data!)</li>
<li><strong>Small sample sizes:</strong> CLT is asymptotic; n ≥ 30 is rule of thumb</li>
<li><strong>Confusing convergence types:</strong> Almost sure ≠ in probability ≠ in distribution</li>
</ol>

</div>

## ML Application Summary

<div class="alert alert-summary">
<h4>🤖 ML Applications Summary</h4>

<table style="width: 100%; border-collapse: collapse; background: white; margin: 10px 0;">
<tr style="background: #e8f5e8;">
<th style="padding: 10px; border: 1px solid #ccc;">Application</th>
<th style="padding: 10px; border: 1px solid #ccc;">How Convergence Concepts Apply</th>
</tr>
<tr>
<td style="padding: 8px; border: 1px solid #ccc;"><strong>Monte Carlo</strong></td>
<td style="padding: 8px; border: 1px solid #ccc;">Error ∝ 1/√n; use CLT for confidence intervals</td>
</tr>
<tr>
<td style="padding: 8px; border: 1px solid #ccc;"><strong>SGD/Mini-batch</strong></td>
<td style="padding: 8px; border: 1px solid #ccc;">Gradient noise ∝ 1/√B; batch size trade-offs</td>
</tr>
<tr>
<td style="padding: 8px; border: 1px solid #ccc;"><strong>Bootstrap</strong></td>
<td style="padding: 8px; border: 1px solid #ccc;">Bootstrap SE ∝ 1/√B; determines # iterations</td>
</tr>
<tr>
<td style="padding: 8px; border: 1px solid #ccc;"><strong>Model Ensembles</strong></td>
<td style="padding: 8px; border: 1px solid #ccc;">Prediction variance ∝ 1/M (M models)</td>
</tr>
<tr>
<td style="padding: 8px; border: 1px solid #ccc;"><strong>A/B Testing</strong></td>
<td style="padding: 8px; border: 1px solid #ccc;">Sample size calculation using CLT</td>
</tr>
<tr>
<td style="padding: 8px; border: 1px solid #ccc;"><strong>Dropout Uncertainty</strong></td>
<td style="padding: 8px; border: 1px solid #ccc;">Average over T passes; uncertainty ∝ 1/√T</td>
</tr>
</table>

</div>

## Key Takeaways

<div class="alert alert-summary">
<h4>🎯 Key Takeaways</h4>

<p><strong>1. Three Types of Convergence</strong></p>
<ul>
<li><strong>In Probability:</strong> P(|Xₙ - X| > ε) → 0 (weakest)</li>
<li><strong>Almost Sure:</strong> P(Xₙ → X) = 1 (stronger)</li>
<li><strong>In Distribution:</strong> CDFs converge (different flavor)</li>
</ul>

<p><strong>2. Law of Large Numbers</strong></p>
<ul>
<li><strong>What:</strong> Sample means converge to population mean</li>
<li><strong>When:</strong> i.i.d. samples with finite mean (and variance for WLLN)</li>
<li><strong>Limitation:</strong> Doesn't specify convergence rate</li>
</ul>

<p><strong>3. Central Limit Theorem</strong></p>
<ul>
<li><strong>What:</strong> Sample means are approximately normal: X̄ₙ ~ N(μ, σ²/n)</li>
<li><strong>Magic:</strong> Works for ANY distribution (with finite variance)</li>
<li><strong>Rate:</strong> Standard error = σ/√n (the famous √n rule!)</li>
<li><strong>Power:</strong> Enables confidence intervals and hypothesis tests</li>
</ul>

<hr>

<h5>🔑 Essential Formulas</h5>

<table style="width: 100%; border-collapse: collapse; background: white; margin: 10px 0;">
<tr style="background: #e3f2fd;">
<th style="padding: 10px; border: 1px solid #ccc;">Concept</th>
<th style="padding: 10px; border: 1px solid #ccc;">Formula</th>
<th style="padding: 10px; border: 1px solid #ccc;">Interpretation</th>
</tr>
<tr>
<td style="padding: 8px; border: 1px solid #ccc;">Sample Mean</td>
<td style="padding: 8px; border: 1px solid #ccc;">X̄ₙ = (X₁+...+Xₙ)/n</td>
<td style="padding: 8px; border: 1px solid #ccc;">Average of n samples</td>
</tr>
<tr>
<td style="padding: 8px; border: 1px solid #ccc;">Expected Value</td>
<td style="padding: 8px; border: 1px solid #ccc;">E[X̄ₙ] = μ</td>
<td style="padding: 8px; border: 1px solid #ccc;">Unbiased estimator</td>
</tr>
<tr>
<td style="padding: 8px; border: 1px solid #ccc;">Variance</td>
<td style="padding: 8px; border: 1px solid #ccc;">Var(X̄ₙ) = σ²/n</td>
<td style="padding: 8px; border: 1px solid #ccc;">Decreases with n</td>
</tr>
<tr>
<td style="padding: 8px; border: 1px solid #ccc;">Standard Error</td>
<td style="padding: 8px; border: 1px solid #ccc;">SE = σ/√n</td>
<td style="padding: 8px; border: 1px solid #ccc;">Std dev of estimate</td>
</tr>
<tr>
<td style="padding: 8px; border: 1px solid #ccc;">95% CI</td>
<td style="padding: 8px; border: 1px solid #ccc;">X̄ ± 1.96·σ/√n</td>
<td style="padding: 8px; border: 1px solid #ccc;">Confidence interval</td>
</tr>
<tr>
<td style="padding: 8px; border: 1px solid #ccc;">Sample Size</td>
<td style="padding: 8px; border: 1px solid #ccc;">n = (z·σ/ε)²</td>
<td style="padding: 8px; border: 1px solid #ccc;">For margin of error ε</td>
</tr>
</table>

</div>

## Useful Links

- [The Central Limit Theorem, Clearly Explained!!! by StatQuest](https://www.youtube.com/watch?v=YAlJCEDH2uY)